# Finetuning des Parser

Hier soll eine Möglichkeit erarbeitet werden, mit der die HTML-Files am besten extrahiert und bereinigt werden können. Damit soll eine reibungslose Weiterverarbeitung der Texte ermöglicht werden.

## Vorüberlegung:

Für unser Projekt sind verschiedene Teile des HTML-Files besonders von Bedeutung:

- Text: Inhalt der HTML-Seite
    - Diese sollte wenn möglich in einzelne Abschnitte gegliedert werden (ermöglicht später die exakte Beantwortung von Fragen)
    - Unwichtige Elemente der Seite (wie Menü-Punkte, die auf jeder Seite erscheinen) gilt es zu filtern
    - Nichtdarstellbare Zeichen gilt es zu filtern
- Überschrift und Unterüberschrift der Paragraphen
    - Wenn man im späteren Verlauf auf eine Frage einen Paragrafen referenzieren möchte, ist es wichtig die zum Paragraph gehörende Unterüberschrift und die Verlinkung dahin zu vermerken
- HTML-File: Der Link zum verwendeten File sollte als Meta-Data vermerkt werden um später zur Seite navigieren zu können
    - (vielleicht kann man noch den Link zur genauen Teil-Überschrift extrahieren)
- (OPTIONAL Links und Bilder: Man könnte die verwendeten Links (href) und den Pfad zu den Bildern mit extrahieren, stellt sich aber die Frage, in wie weit das relevant ist) 

### Benötigte Pakete:

In [ ]:
from bs4 import BeautifulSoup
import glob
import re
import csv
import itertools

(Ich habe Probleme in kedro jupyter notebooks das Paket pandas zu verwenden, mit diesem workaround ist es mir aber möglich)

In [ ]:
import sys
!{sys.executable} -m pip install pandas

In [ ]:
import pandas as pd

### Auslesen von HTML-Elementen

Hier wird zunächst nur repräsentativ ein kleiner Teil der Dokumentationsdateien ausgelesen (alle Seiten zum Punkt "Bedienung").
Die ausgelesenen Daten werden in einem CSV-File gespeichert:

|Title| Text| Source |
|:----|:----|:-------|
| h1  |...  | file1  |
| h2  |...  | file1  |
| h1  |...  | file2  |
| h2  |...  | file2  |
| h2  |...  | file2  |
| ... |...  | ...    |

In [114]:
path = 'path/to/*.html'
files = glob.glob(path)
all_data = []
for file in files:
    f = open(file, 'r', encoding = "utf-8")
    name = file
    soup = BeautifulSoup(f.read(),'html.parser')
    h1 = get_body(soup, get_paraName(soup,"h1"),name)
    h2 = get_body(soup, get_paraName(soup,"h2"),name)
    all_data.append(h1)
    all_data.append(h2)
    f.close()
    #write the data
flat_list = list(itertools.chain(*all_data))
dataframe = pd.DataFrame(flat_list, columns = ["Title","Source","Hash","Text"])
dataframe.to_csv("test_cleanFiles.csv")

#### Erzeugen des Text-Body Elements

Im Div-Block mit der Klasse "document" wird der Inhalt der Seite umschlossen (ausgeschlossen das Seiten Menü, Header und Fußzeile)

Die Unterteilungen in der Seite erfolgen durch kleinere Div-Blöcke der Klasse "section".
Die id der Div-Blöcke entspricht der Überschrift (h1 oder h2).
Daher werden nun Anhand der Überschriften zunächst die Div-Blöcke gesucht und anschließend die p-Blöcke, Auflistungen (ul) und Tabellen (table) ausgelesen. Diese werden als einzelne Elemente einer Liste abgespeichert.

Zudem wird noch ein eindeutiger Hashwert auf Basis der aktuellen Überschrift erstell, um später die Elemente eindeutig ausweisen zu können.

In [110]:
def get_body(soup_elem,liste_headlines,name):
    ordnen = []
    for h in liste_headlines:
        block = soup_elem.find("div",{"id":h.lower()})
        elems = (block.findChildren(("p","ul","table"),recursive =False))
        for elem in elems:
            text = re.sub(" \n ", " ", elem.get_text().strip(), flags=re.M)
            text = re.sub("\n ", " ", text.strip(), flags=re.M)
            text = re.sub(" \n", " ", text.strip(), flags=re.M)
            text =re.sub("\n", " ", text.strip(), flags=re.M)
            ordnen.append((text, h, name, hash(h)))
    text_frame = pd.DataFrame(ordnen, columns = ["Text","Head","Link","Hash"])
    text_merge = text_frame.groupby(['Head',"Link","Hash"],sort=False)['Text'].apply(list)
    
    final = text_merge.reset_index().values.tolist()
    return final

#### Erzeugen des Text-Body Elements (VERALTET)

Aktuell wird der Teil für h1-Überschriften getrennt betrachtet, da hier spezielle Parent-children Bezeihungen verwendet werden
um den Text der nur zur h1-Überschrift gehört zu extrahieren.

Da die Funktion get_body() zur korrekten Teilung der h2 Überschriften angepasst werden musste, ist diese Funktion nun hinfällig.

In [ ]:
def get_body_h1(soup_elem,liste_headlines ,name):
    clean_para=[]
    ordnen = []
    for h1 in liste_headlines:
        block = soup_elem.find("div",{"id":h1.lower()})
        for para in block.findChildren(("p","ul"), recursive=False):
            clean_para.append(re.sub("\n", " ", para.get_text().strip(), flags=re.M))
        ordnen.append((h1, clean_para,name))
    return ordnen

#### Finde Kapitel

Es müssen die Kapitelnamen und die Namen der Unterkapitel für jede Datei gesucht werden. Nur so kann später der Inhalt den Überschriften zugeordnet werden.
Ein zu der Überschrift gehörender Block, trägt meist den Namen der dazugehördenen Überschrift. Ausnahmen entstehen durch Sonderzeichen.
Da die h1-Überschriften anders zu behandeln sind als h2-Überschriften muss als Parameter übergeben werden welcher Typ von Überschrift (headline "h1" oder "h2") gerade betrachtet wird.

Überlegung: Zur Zeit werden die Namen der Überschriften noch umständlich angepasst, um als id der zugehörigen Div-Blöcke verwendet zu werden. Vielleicht gibt es da einen besseren Work-Around.

In [ ]:
def get_paraName(soup_elem, headline):
    all_unter = []
    for unterkap in soup_elem.find_all(headline):
        sauber = re.sub('\uf0c1','', unterkap.get_text())
        string = sauber.replace("(","").replace(")","")
        sauber = re.sub('\W+',' ',string)
        sauber = re.sub('ü','u',sauber)
        sauber = re.sub('ä','a',sauber)
        sauber = re.sub('ö','o',sauber)
        sauber = re.sub('Ü','U',sauber)
        sauber = re.sub('Ä','A',sauber)
        sauber = re.sub('Ö','O',sauber)
        sauber = sauber.replace("ECU TEST","Produktname")
        all_unter.append(re.sub(' ','-', sauber))
    return all_unter

#### Finde Links (Optional)

In [ ]:
def get_link(soup_elem):
    all_link = []
    for link in soup_elem.find_all('a'):
        all_link.append(link.get('href'))
    return all_link 

#### Finde Bilder (Optional)

In [ ]:
def get_pic(soup_elem):
    all_bild=[]
    for bild in soup_elem.find_all('img'):
        all_bild.append(bild.get('src'))
    return all_bild

### Test-Bereich für einzelnen Funktionen:

In diesem Abschnitt werden die nötigen Funktionen getestet und bearbeitet, die später im oberen Teil zu einem korrekten Ergebnis zusammengeführt werden.

In [ ]:
HTMLFile = open("path/to/file.html", "r", encoding = "utf-8")
index = HTMLFile.read()
S = BeautifulSoup(index,'html.parser')

Idee: Alle Paragrafen sind als Div Elemente durch die entsprechende Überschrift (h2) als ID versehen

    - Sammle alle h2-Überschriften einer Seite
    - Rufe anhand der Überschriften die zugehörigen h2 Divs auf
    - extrahiere aus ihnen die p-Blöcke mit dem eigentlichen Inhalte der Seite

In [ ]:
liste_h2 = get_paraName(S,"h2")
print(liste_h2)

Für jedes Unterkapitel (h2-Überschrift) finde alle P-Blöcke und übernimm den bereinigten Textinhalt in eine Liste. Anschließend ordne diese Liste der entsprechenden Überschrift zu.

Aufbau Liste:

| Paragrafen   | Inhalt                               |
|:----------- | :---------------------------------- |
|Überschrift 1 | (Textblock p1), (Textblock p2), .... |
|Überschrift 2 | (Textblock p1), (textblock p2), .... |

In [ ]:
clean_para=[]
ordnen = []
for h2 in liste_h2:
    block = S.find("div",{"id":h2.lower()})
    for para in block.findChildren(("p","ul"),recursive=False):
        clean_para.append(re.sub("\n", " ", para.get_text().strip(), flags=re.M))
    ordnen.append((h2, clean_para))
print(ordnen[2])
    

### Problem (gelöst durch Lösungsansatz 1)

- Aktuell wird für die Überschrift h1 noch der gesamte Seiteninhalt vermerkt -> man muss also überlegen wie man hier nur den Text bis zur ersten h2-Überschrift extrahieren kann

## Lösungsansatz 1 für h1 Überschrift

Da h1 ja alle anderen Überschriften mit umfasst, muss irgendwie gefiltert werden, das dieser Text nicht mit übernommen wird. 
Das Problem könnte über eine children-parent Beziehung gelöst werden:

- Für die Überschrift h1 Wähle alle p-Blöcke die direkte Kinder des h1-Div-Blocks sind

In [ ]:
block = S.find("div",{"id":"suche"})

In [ ]:
children = block.findChildren(("p","ul"), recursive=False)

Da für h1-Überschriften nur die direkten Kinder zum Auslesen interessant sind, ist es notwendig für diese eine seperate Funktion zu definieren (get_body_h1)

In [ ]:
test =get_body_h1(S, get_paraName(S,"h1"),"Source.html")

In [ ]:
print(test)

In [ ]:
all_data=[]
path = 'path/to/*.html'
files = glob.glob(path)
for file in files:
    name = file
    f = open(file, 'r', encoding = "utf-8")
    soup = BeautifulSoup(f.read(),'html.parser')
    h1 = get_body_h1(soup, get_paraName(soup,"h1"),"Source.html")
    h2 = get_body(soup, get_paraName(soup,"h2"),"Source.html")
    all_data.append(h1)
    all_data.append(h2)
    f.close()

#### Evaluieren der Ausgabe und Visualisierung als DataFrame

In [ ]:
flat_list = list(itertools.chain(*all_data))
dataframe = pd.DataFrame(flat_list, columns = ["Title","Text","Source"])

In [ ]:
print(dataframe.head())

#### Fazit zum Lösungsversuch:

- Methode bringt sowohl h1 als auch h2-Überschriften und ihren Inhalt zusammen
- Ausgelesene Daten sind weitestgehend bereinigt

## Fehler 2:
Die h2-Überschriften geben aktuell alle nur den gesamten Inhalt aller Blöcke wieder anstatt nur ihren eigenen.

### Weitere Ergänzungen:

Es wär praktisch für die Weiterverwendung der Daten, einzelne Inhalte später eindeutig auseinander halten zu können. Hierzu würde es sich anbieten die Daten über einen Hash-Wert eindeutig auszuweisen. Da die Daten eindeutige Überschriften besitzen (?) kann dieser als Hash-Wert verwendet werden.

### Lösungsansatz 2:

- da aktuell der gesamte Inhalt pro Zwischenüberschrift übernommen wird, muss hier auf die Child-Parent Beziehung geachtet werden
- außerdem ist es wichtig die Text-Elemente noch getrennt betrachten zu können, daher werden sie als Elemente eines Liste gespeichert

In [ ]:
liste = []
for h2 in liste_h2:
    block = S.find("div",{"id":h2.lower()})
    elems = (block.findChildren(("p","ul","table"),recursive =False))
    for elem in elems:
        text =re.sub("\n", " ", elem.get_text().strip(), flags=re.M)
        liste.append((text, h2))
frame = pd.DataFrame(liste, columns = ["Text","Head"])
group_frame = frame.groupby('Head',sort=False)['Text'].apply(list)
print(group_frame[1])

### Ergänzung um Hash-Wert

- Einfügen der oben getesteten Version
- Ergänzen um einen Hash-Wert

In [111]:
test = get_body(S, liste_h2, "test")

In [112]:
test_frame = pd.DataFrame(test,columns=["Head","Link","Hash","Body"])

In [113]:
print(test_frame.head())

                      Head  Link                 Hash  \
0             Terminologie  test  2232245079872126677   
1            Konfiguration  test  -226203862521301321   
2  Arbeiten-mit-Attributen  test  8594958551157606788   
3    Arbeiten-mit-Packages  test  3299522519948488173   
4   Arbeiten-mit-Projekten  test -4508554425207993390   

                                                Body  
0  [ECU-TEST und die jeweiligen Testmanagement-Sy...  
1  [Vor der ersten Verwendung muss die Schnittste...  
2  [Packages und Projekte können in ECU-TEST mit ...  
3  [Mit ECU-TEST kann man Packages als Testskript...  
4  [ECU-TEST kann Projekte als Testsuiten in das ...  
